In [60]:
import pandas as pd

In [61]:
path = '/Users/david/Documents/phd/notebooks/LTRData/Fold1/'

train_file  = path + 'train.txt'
test_file = path + 'test.txt'

In [63]:
train_df = pd.read_csv(train_file, sep=' ', header=None, dtype=str)
test_df = pd.read_csv(test_file, sep=' ', header=None, dtype=str)

In [64]:
train_df.rename(columns={0: 'label', 1: 'query_id'}, inplace=True)
test_df.rename(columns={0: 'label', 1: 'query_id'}, inplace=True)

In [65]:
train_df.head()

,label,query_id,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,2,qid:1,1:3,2:3,3:0,4:0,5:3,6:1,7:1,8:0,...,128:11089534,129:2,130:116,131:64034,132:13,133:3,134:0,135:0,136:0,NaN
1,2,qid:1,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:11089534,129:2,130:124,131:64034,132:1,133:2,134:0,135:0,136:0,NaN
2,0,qid:1,1:3,2:0,3:2,4:0,5:3,6:1,7:0,8:0.666667,...,128:3,129:1,130:124,131:3344,132:14,133:67,134:0,135:0,136:0,NaN
3,2,qid:1,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:11089534,129:13,130:123,131:63933,132:1,133:3,134:0,135:0,136:0,NaN
4,1,qid:1,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:5,129:7,130:256,131:49697,132:1,133:13,134:0,135:0,136:0,NaN


In [66]:
train_df.drop(138, axis=1, inplace=True)
test_df.drop(138, axis=1, inplace=True)

In [67]:
def extract_value_after_colon(df):

    #Quebra as strings por ':' e retorna apenas o valor apos os dois pontos 
    df.iloc[:,1:] = df.iloc[:,1:].applymap(lambda x: x.split(':')[1] if isinstance(x, str) else x)
    return df

In [68]:
train_df = extract_value_after_colon(train_df)
test_df = extract_value_after_colon(test_df)

In [70]:
def split_labels_features_queryIds(df):
    
    #Separando

    #Rotulos
    labels = df['label'].astype(int)

    #Quantidade de documentos para cada consulta
    query_id = df.groupby("query_id")["query_id"].count().to_numpy()

    #Caracteristicas
    features = df.iloc[:, 2:].astype(float)

    return labels, features, query_id

In [71]:
y_train, X_train, qids_train = split_labels_features_queryIds(train_df)
y_test, X_test, qids_test = split_labels_features_queryIds(test_df)

In [72]:
import lightgbm as lgb

# Definindo modelo e hiperparametros
ranker = lgb.LGBMRanker(
                    objective="lambdarank",
                    boosting_type = "gbdt",
                    n_estimators = 5,
                    importance_type = "gain",
                    num_leaves = 10,
                    learning_rate = 0.01,
                    max_depth = -1)

# Treinando o modelo
ranker.fit(
      X=X_train,
      y=y_train,
      group=qids_train)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.090243 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 25637
[LightGBM] [Info] Number of data points in the train set: 723412, number of used features: 136


LGBMRanker(importance_type='gain', learning_rate=0.01, n_estimators=5,
           num_leaves=10, objective='lambdarank')

In [73]:
import numpy as np

def ndcg(labels, predictions):
    # Calcula Normalized Discounted Cumulative Gain (NDCG)
    labels = np.asarray(labels)
    predictions = np.asarray(predictions)
    
    # Ordena as predições em ordem descrente
    sorted_indices = np.argsort(predictions)[::-1]
    
    # Retorna para seus rotulos associados
    sorted_labels = labels[sorted_indices]
    
    # Calcula o Discounted Cumulative Gain (DCG)
    dcg = np.sum((2 ** sorted_labels - 1) / np.log2(np.arange(2, len(labels) + 2)))
    
    # Orden os rotulos em ordem descedente para calcular o IDCG
    ideal_labels = np.sort(labels)[::-1]
    ideal_dcg = np.sum((2 ** ideal_labels - 1) / np.log2(np.arange(2, len(labels) + 2)))
    
    # Calcula NDCG pela normalizacao do DCG pelo IDCG
    ndcg = dcg / ideal_dcg if ideal_dcg != 0 else 0
    
    return ndcg

pred_test = ranker.predict(X_test, raw_score=False)
ndcg(y_test, pred_test)

0.8752754784492303